# Unit Testing : Making sure your bugs don't come back

Unity testing allows to ensure that a given software behaves in the
correct way, at least for the cases one is testing. Once a function is
written (or even before in TTD) or a bug is fixed, it is necessary to
write a test that ensures the function to work properly in limit cases
or the bug to not reappear in the future. There are several levels
associated with [unit testing
.](https://en.wikipedia.org/wiki/Unit_testing)

In this unit we will learn the general philosophy behind it and a couple
of tools to implement very basic tests, althoguh the list of [testing
frameworks](https://en.wikipedia.org/wiki/List_of_unit_testing_frameworks)
is very large. Furthermore, modularization will be very important, so
you must have a clear understanding on how to split some given code into
headers, source files, and how to compile objects and then link them
using the linker, hopefully through a Makefile.

## Catch2

Our goal here is to learn to use
[catch2](https://github.com/catchorg/Catch2/blob/master/docs/tutorial.md#top)
to test a very simple function extracted from their tutorial. Later we
will modularize the code to practice that and write a useful `Makefile`.

### Install catch2

We will install catch2 using `spack`:

``` shell
spack install catch2
```

### Tutorial example

This is the file example extracted from `catch2` tutorial.

``` cpp
#define CATCH_CONFIG_MAIN  // This tells Catch to provide a main() - only do this in one cpp file
#include "catch2/catch.hpp"
//#include "catch.hpp"

unsigned int Factorial( unsigned int number ) {
  return number <= 1 ? number : Factorial(number-1)*number;
}

TEST_CASE( "Factorials are computed", "[factorial]" ) {
  REQUIRE( Factorial(1) == 1 );
  REQUIRE( Factorial(2) == 2 );
  REQUIRE( Factorial(3) == 6 );
  REQUIRE( Factorial(10) == 3628800 );
}
```

Catch is header only. Compile this as

``` shell
g++ example_test.cpp
```

then run the executable and the tests info will be presented on the
screen.

Can you see if there is a bug in the factorial code? fix it and add a
test `REQUIRE` in the same file.

### Code modularization

Modularization allows us to split the implementation, the interface, and
the test.

To modularize the code we need to take the only function here,
`factorial`, and split it into a header and source file:

-   header file:

    ``` cpp
    #pragma once
    int factorial(int n);
    ```

-   Implementation file:

    ``` cpp
    #include "factorial.h"

    int factorial(int number)
    {
    return number <= 1 ? number : factorial(number-1)*number;
    }
    ```

Clearly this `factorial.cpp` file cannot be compiled into and executable
since it does no contain a main function. But it can be used to
construct and object file (flag `-c` on the compiler) to later link with
other object files.

Now we create a main file that will call the `factorial` function:

``` cpp
#include <iostream>
#include "factorial.h"

int main(void)
{
  std::cout << factorial(4) << std::endl;
  return 0;
}
```

And then compile it as

``` shell
g++ -c factorial.cpp # creates factorial.o that can be used for the testing
g++ -c factorial_main.cpp # creates main_factorial.o
g++ factorial.o factorial_main.o -o main_factorial.x # links and creates executable
./main_factorial.x
```

### Writing the test

Let's first write the test

``` cpp
#define CATCH_CONFIG_MAIN  // This tells Catch to provide a main() - only do this in one cpp file
#include "catch2/catch.hpp"

#include "factorial.h"

TEST_CASE( "Factorials are computed", "[factorial]" ) {
    //REQUIRE( factorial(0) == 1 );
    REQUIRE( factorial(1) == 1 );
    REQUIRE( factorial(2) == 2 );
    REQUIRE( factorial(3) == 6 );
    REQUIRE( factorial(10) == 3628800 );
}
```

Now let's run the test using the modularized code. First, let's load
`catch`

``` shell
spack load catch2
```

Compile and execute as

``` shell
g++ -c factorial.cpp
g++ -c factorial_test.cpp
g++ factorial.o factorial_test.o -o factorial_test.x
./factorial_test.x
```

Now you can test or run-with-not-tests independently.

### Makefile

To compile this and future tests, is useful to implement all in a
`Makefile`

``` makefile

SHELL:=/bin/bash

all: factorial_main.x

test: factorial_test.x
    ./$<

%.x: %.o factorial.o
    source $$HOME/repos/spack/share/spack/setup-env.sh; \
    spack load catch2; \
    g++ $$(pkg-config --cflags catch2) $^ -o $@

%.o: %.cpp
    source $$HOME/repos/spack/share/spack/setup-env.sh; \
    spack load catch2; \
    g++ $$(pkg-config --cflags catch2) -c $<

clean:
    rm -f *.o *.x
```

## google test

Google test is a famous and advance unit framework that goes well beyond
of what is shown here. You are invited to follow the
[docs](https://github.com/google/googletest/blob/master/googletest/docs/primer.md)
to learn more.

### Installation

Again, we will use spack

``` shell
spack install googletest
mkdir googletest
```

### Example

This is an example, already modularized.

-   Factorial and isprime header:

``` cpp
#ifndef GTEST_SAMPLES_SAMPLE1_H_
#define GTEST_SAMPLES_SAMPLE1_H_

// Returns n! (the factorial of n).  For negative n, n! is defined to be 1.
int Factorial(int n);

//// Returns true if and only if n is a prime number.
bool IsPrime(int n);

#endif  // GTEST_SAMPLES_SAMPLE1_H_
```

-   Source file

``` cpp
#include "factorial.h"

// Returns n! (the factorial of n).  For negative n, n! is defined to be 1.
    int Factorial(int n) {
    int result = 1;
    for (int i = 1; i <= n; i++) {
        result *= i;
    }

    return result;
  }

// Returns true if and only if n is a prime number.
bool IsPrime(int n) {
    // Trivial case 1: small numbers
    if (n <= 1) return false;

    // Trivial case 2: even numbers
    if (n % 2 == 0) return n == 2;

    // Now, we have that n is odd and n >= 3.

    // Try to divide n by every odd number i, starting from 3
    for (int i = 3; ; i += 2) {
        // We only have to try i up to the square root of n
        if (i > n/i) break;

        // Now, we have i <= n/i < n.
        // If n is divisible by i, n is not prime.
        if (n % i == 0) return false;
    }

    // n has no integer factor in the range (1, n), and thus is prime.
    return true;
}
```

-   Test source file (to be compiled as an object)

``` cpp
#include <limits.h>
#include "factorial.h"
#include "gtest/gtest.h"
namespace {
    // Tests factorial of negative numbers.
    TEST(FactorialTest, Negative) {
        // This test is named "Negative", and belongs to the "FactorialTest"
        // test case.
        EXPECT_EQ(1, Factorial(-5));
        EXPECT_EQ(1, Factorial(-1));
        EXPECT_GT(Factorial(-10), 0);
    }
    // Tests factorial of 0.
    TEST(FactorialTest, Zero) {
        EXPECT_EQ(1, Factorial(0));
    }

// Tests factorial of positive numbers.
    TEST(FactorialTest, Positive) {
        EXPECT_EQ(1, Factorial(1));
        EXPECT_EQ(2, Factorial(2));
        EXPECT_EQ(6, Factorial(3));
        EXPECT_EQ(40320, Factorial(8));
    }

    // Tests negative input.
    TEST(IsPrimeTest, Negative) {
        // This test belongs to the IsPrimeTest test case.

        EXPECT_FALSE(IsPrime(-1));
        EXPECT_FALSE(IsPrime(-2));
        EXPECT_FALSE(IsPrime(INT_MIN));
    }

// Tests some trivial cases.
    TEST(IsPrimeTest, Trivial) {
        EXPECT_FALSE(IsPrime(0));
        EXPECT_FALSE(IsPrime(1));
        EXPECT_TRUE(IsPrime(2));
        EXPECT_TRUE(IsPrime(3));
    }

// Tests positive input.
    TEST(IsPrimeTest, Positive) {
        EXPECT_FALSE(IsPrime(4));
        EXPECT_TRUE(IsPrime(5));
        EXPECT_FALSE(IsPrime(6));
        EXPECT_TRUE(IsPrime(23));
    }
}
```

-   Main google test file

``` cpp
#include <cstdio>
#include "gtest/gtest.h"

GTEST_API_ int main(int argc, char **argv) {
    printf("Running main() from %s\n", __FILE__);
    testing::InitGoogleTest(&argc, argv);
    return RUN_ALL_TESTS();
}
```